In [1]:
from pygeoinf.interval import (Lebesgue, IntervalDomain, BoundaryConditions,
                               IntegrationConfig, ParallelConfig,
                               Laplacian, BesselSobolevInverse)
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import cholesky, LinAlgError

In [5]:
int_cfg = IntegrationConfig(method='simpson', n_points=1000)
parallel_cfg = ParallelConfig(
    enabled=True,
    n_jobs=16
)
f_doms = [IntervalDomain(0, 1, boundary_type=boundary_type, open_epsilon=1e-9) for boundary_type in ['open', 'closed', 'left_open', 'right_open']]
for f_dom in f_doms:
    print(f_dom)

    N = 50
    M = Lebesgue(N, f_dom, basis='sine', integration_config=int_cfg, parallel_config=parallel_cfg)

    bcs = BoundaryConditions(bc_type='dirichlet')
    alpha = 1
    k, s = 1, 1
    L = Laplacian(M, bcs, alpha, method='spectral', dofs=100, integration_config=int_cfg, n_samples=1024)
    C = BesselSobolevInverse(M, M, k, s, L, dofs=100, integration_config=int_cfg, n_samples=1024)

    C_mat = C.matrix(dense=True, parallel=True, n_jobs=16)

    print(np.max(np.abs(C_mat - C_mat.T)))  # Check symmetry

    print(np.allclose(C_mat, C_mat.T, rtol=1e-5))  # Check symmetry

    C_mat_sym = (C_mat + C_mat.T)/2

    try:
        cholesky(C_mat_sym, lower=False)   # or np.linalg.cholesky(A)
        print("PD (Cholesky succeeded)")
    except LinAlgError:
        print("Not PD (Cholesky failed)")

(0.0, 1.0)
6.366082132414759e-10
True
PD (Cholesky succeeded)
[0.0, 1.0]
2.553216445077959e-17
True
PD (Cholesky succeeded)
(0.0, 1.0]
6.366069991513545e-10
True
PD (Cholesky succeeded)
[0.0, 1.0)
6.366069633407822e-10
True
PD (Cholesky succeeded)
